In [37]:
import requests
import pandas as pd 
pd.set_option('mode.chained_assignment', None)

In [38]:
df = pd.read_csv("./rawData.csv") 

In [39]:
def setVesselType(number):
    type = '';
    if(number == '6_'):
        type = 'passenger';
    if(number == '7_'):
        type = 'cargo';
    if(number == '8_'):
        type = 'tanker';
    if(number == '3_'):
        type = 'tug';
    if(number == '0_'):
        type = 'undefined';
    if(number == '9_'):
        type = 'yacht';
    if(number == '4_'):
        type = 'high speed';
    if(number == '10'):
        type = 'fishing';
    if(number == '13'):
        type = 'navigation aid';
    return type;

In [40]:
#clean VesselType
for i in df.index:
    number = df['VesselType'][i][11:13]
    df['VesselType'][i] = setVesselType(number)

In [41]:
print(df) 

     Unnamed: 0      Event              Time                    Port  \
0             0  Departure  2019-12-08 15:31               AMSTERDAM   
1             1    Arrival  2019-12-08 15:31          CALETA PUELCHE   
2             2    Arrival  2019-12-08 15:31                 VAXHOLM   
3             3    Arrival  2019-12-08 15:31       ROTTERDAM CENTRUM   
4             4    Arrival  2019-12-08 15:31              RAS LAFFAN   
5             5    Arrival  2019-12-08 15:31                  SLOVAG   
6             6    Arrival  2019-12-08 15:31           FREEPORT (TX)   
7             7    Arrival  2019-12-08 15:31               GOOD HOPE   
8             8    Arrival  2019-12-08 15:31             NEW ORLEANS   
9             9    Arrival  2019-12-08 15:31                    TYNE   
10           10    Arrival  2019-12-08 15:31               OSTERMOOR   
11           11    Arrival  2019-12-08 15:31                PATILLOS   
12           12  Departure  2019-12-08 15:31       ROTTERDAM CEN

In [42]:
#create set of vessels
vessels = set(df.Vessel)

In [43]:
#create new df for linked values (arrival and departure)
timetable = pd.DataFrame([], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])

In [44]:
#vessel filter and combine arrival and deaprture
for v in vessels:
    allEntriesOfVessel = df[df['Vessel'].isin([v])]
    depFirst = False
    for index, row in allEntriesOfVessel[::-1].iterrows():
        arrivalPort, arrivalTime = "", ""
        
        if(allEntriesOfVessel.Event[index] == "Departure"):
            departurePort = (allEntriesOfVessel.Port[index])
            departureTime = (allEntriesOfVessel.Time[index])
            vessel = (allEntriesOfVessel.Vessel[index])
            vesselType = (allEntriesOfVessel.VesselType[index])
            depFirst = True
        if(allEntriesOfVessel.Event[index] == "Arrival" and depFirst):
            arrivalPort = (allEntriesOfVessel.Port[index])
            arrivalTime = (allEntriesOfVessel.Time[index])
            depFirst = False

            if(departurePort != "" or departureTime != "" or vessel != "" or vesselType != "" or arrivalPort != "" or arrivalTime!= ""):
                newEntry = pd.DataFrame([[departurePort, departureTime, arrivalPort, arrivalTime, vessel, vesselType]], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])
                timetable = timetable.append(newEntry, ignore_index = True)
            

In [46]:
#drop all rows that have any NaN values
timetable = timetable.dropna()

In [48]:
print(timetable)
timetable.to_csv("timetable.csv")

          DeparturePort     DepartureTime        ArrivalPort  \
0               ANTWERP  2019-12-08 15:23           HEMIKSEM   
1             KOLHATTAN  2019-12-08 15:20          SVANESUND   
2             MONGSTAD   2019-12-08 15:21             SLOVAG   
3        PETIT COURONNE  2019-12-08 15:21     VAL-DE-LA-HAYE   
4     ROTTERDAM CENTRUM  2019-12-08 15:21  ROTTERDAM CENTRUM   
5     ROTTERDAM CENTRUM  2019-12-08 15:27  ROTTERDAM CENTRUM   
6            LA BOUILLE  2019-12-08 15:24         LA BOUILLE   
7                LISBOA  2019-12-08 15:23             ALMADA   
8            HJELMELAND  2019-12-08 15:21             NESVIK   
9                  HALS  2019-12-08 15:20               HALS   
10  ROTTERDAM WAALHAVEN  2019-12-08 15:23           SCHIEDAM   
11             GELKENES  2019-12-08 15:27        SCHOONHOVEN   
12            AMSTERDAM  2019-12-08 15:23            ZAANDAM   
13               PERAMA  2019-12-08 15:20           SALAMINA   
14            GEITANGER  2019-12-08 15:2